In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/image-matching-challenge-2025/sample_submission.csv
/kaggle/input/image-matching-challenge-2025/train_thresholds.csv
/kaggle/input/image-matching-challenge-2025/train_labels.csv
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et004.png
/kaggle/input/image-matching-challenge-2025/test/ETs/outliers_out_et003.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et006.png
/kaggle/input/image-matching-challenge-2025/test/ETs/et_et004.png
/kaggle/input/image-matching-challenge-2025/test/ETs/et_et002.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et009.png
/kaggle/input/image-matching-challenge-2025/test/ETs/outliers_out_et001.png
/kaggle/input/image-matching-challenge-2025/test/ETs/et_et006.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et005.png
/kaggle/input/image-matching-challenge-2025/test/ETs/another_et_another_et008.png
/kaggle/input/image-matching-challenge-2025/test

In [2]:
import os
import torch

print("--- 1. PREPARATION: CLEANING & CREATING DIRS ---")
# Clean up any previous bad runs to ensure a fresh start
!rm -rf offline_wheels weights my_offline_dependencies.zip

# Create directory structure
os.makedirs("offline_wheels", exist_ok=True)
os.makedirs("weights/dinov2", exist_ok=True)
os.makedirs("weights/roma", exist_ok=True)

print("\n--- 2. DOWNLOADING STANDARD LIBRARIES (PyPI) ---")
# Only standard pip packages go here. 
# REMOVED: hloc, lightglue (they fail here)
!pip download pycolmap kornia \
    --dest offline_wheels \
    --prefer-binary

print("\n--- 3. BUILDING RESEARCH LIBRARIES (GitHub) ---")
# We build these from source because they are research code not always on PyPI
!pip wheel git+https://github.com/cvg/Hierarchical-Localization.git --w offline_wheels
!pip wheel git+https://github.com/cvg/LightGlue.git --w offline_wheels
!pip wheel git+https://github.com/google-research/omniglue.git --w offline_wheels
!pip wheel git+https://github.com/Parskatt/RoMa.git --w offline_wheels

print("\n--- 4. DOWNLOADING MODEL WEIGHTS ---")

# A. DINOv2
print("Downloading DINOv2...")
dino = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
torch.save(dino.state_dict(), "weights/dinov2/dinov2_vitl14.pth")

# B. RoMa (The Critical Fix)
print("Downloading RoMa (from Parskatt/storage)...")
# Using the 'storage' repo URL which bypasses the 404/401 errors on the main repo
!wget "https://github.com/Parskatt/storage/releases/download/roma/roma_outdoor.pth" \
    -O weights/roma/roma_outdoor.pth

print("\n--- 5. VERIFICATION ---")
# Check RoMa file size to prevent bad zips
roma_size = os.path.getsize("weights/roma/roma_outdoor.pth") / (1024 * 1024)
print(f"RoMa Weight Size: {roma_size:.2f} MB")

if roma_size < 100:
    raise Exception("CRITICAL ERROR: RoMa weights failed to download (File too small). Do not submit!")

print("\n--- 6. ZIPPING FINAL ARTIFACT ---")
!zip -q -r my_offline_dependencies.zip offline_wheels weights

print("\n✅ SUCCESS! Go to the 'Output' tab and download 'my_offline_dependencies.zip'")

--- 1. PREPARATION: CLEANING & CREATING DIRS ---

--- 2. DOWNLOADING STANDARD LIBRARIES (PyPI) ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitl14_pretrain.pth
100%|██████████| 1.13G/1.13G [00:03<00:00, 342MB/s]


--2025-11-20 16:13:51--  https://github.com/Parskatt/storage/releases/download/roma/roma_outdoor.pth
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/483979090/2cd9a374-9a4a-477c-a8d6-142129f1b39e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-20T16%3A57%3A49Z&rscd=attachment%3B+filename%3Droma_outdoor.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-20T15%3A56%3A53Z&ske=2025-11-20T16%3A57%3A49Z&sks=b&skv=2018-11-09&sig=oUZyy7EQ5wGzcZpPf0cTfKttOclyFzAKCKJyNyUwIsU%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzY1ODgzMSwibmJmIjoxNzYzNjU1MjMxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iL